In [8]:
# package imports
from dotenv import load_dotenv
import numpy as np
import os
import pandas as pd
import requests

In [11]:
# read environment variables
load_dotenv()
SPOTIFY_API_BASE_URL = os.getenv('SPOTIFY_API_BASE_URL')

In [7]:


def get_access_token():
    res = requests.post(f"{SPOTIFY_API_BASE_URL}/token",
        headers = { "Content-Type" : "application/x-www-form-urlencoded" },
        data = {
            "grant_type": "client_credentials",
            "client_id": f"<client_id>",
            "client_secret": "<client_secret>"
        }
    )